In [75]:
import numpy as np
import pandas as pd
import scipy
from math import sqrt
import matplotlib.pyplot as plt

#Classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn import linear_model

#model metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score

#cross validation
from sklearn.model_selection import train_test_split
pd.set_option('display.max_columns', None)

from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE

rawData = pd.read_csv('C:/Users/Jeremy/Desktop/UT Data Analytics Course/5/C5T1/default of credit card clients.csv', index_col = 0, header = 1)
rawData.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
ID,,,,,,,,,,,,,,,,,,,,,,,,
1,20000,2,2,1,24,2,2,-1,-1,-2,-2,3913,3102,689,0,0,0,0,689,0,0,0,0,1
2,120000,2,2,2,26,-1,2,0,0,0,2,2682,1725,2682,3272,3455,3261,0,1000,1000,1000,0,2000,1
3,90000,2,2,2,34,0,0,0,0,0,0,29239,14027,13559,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
4,50000,2,2,1,37,0,0,0,0,0,0,46990,48233,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
5,50000,1,2,1,57,-1,0,-1,0,0,0,8617,5670,35835,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [76]:
rawData.dtypes

LIMIT_BAL                     int64
SEX                           int64
EDUCATION                     int64
MARRIAGE                      int64
AGE                           int64
PAY_0                         int64
PAY_2                         int64
PAY_3                         int64
PAY_4                         int64
PAY_5                         int64
PAY_6                         int64
BILL_AMT1                     int64
BILL_AMT2                     int64
BILL_AMT3                     int64
BILL_AMT4                     int64
BILL_AMT5                     int64
BILL_AMT6                     int64
PAY_AMT1                      int64
PAY_AMT2                      int64
PAY_AMT3                      int64
PAY_AMT4                      int64
PAY_AMT5                      int64
PAY_AMT6                      int64
default payment next month    int64
dtype: object

In [77]:
rawData.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
ID,,,,,,,,,,,,,,,,,,,,,,,,
1,20000,2,2,1,24,2,2,-1,-1,-2,-2,3913,3102,689,0,0,0,0,689,0,0,0,0,1
2,120000,2,2,2,26,-1,2,0,0,0,2,2682,1725,2682,3272,3455,3261,0,1000,1000,1000,0,2000,1
3,90000,2,2,2,34,0,0,0,0,0,0,29239,14027,13559,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
4,50000,2,2,1,37,0,0,0,0,0,0,46990,48233,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
5,50000,1,2,1,57,-1,0,-1,0,0,0,8617,5670,35835,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [78]:
rawData.loc[rawData['EDUCATION'] > 3, 'EDUCATION'] = 0

In [79]:
#features
features = rawData.iloc[:,:5]
features = pd.get_dummies(data=features, columns=['EDUCATION', 'MARRIAGE'], prefix=['edu','mar'])
features.SEX = features.SEX.replace(1,0)
features.SEX = features.SEX.replace(2,1)
features


,LIMIT_BAL,SEX,AGE,edu_0,edu_1,edu_2,edu_3,mar_0,mar_1,mar_2,mar_3
ID,,,,,,,,,,,
1,20000,1,24,0,0,1,0,0,1,0,0
2,120000,1,26,0,0,1,0,0,0,1,0
3,90000,1,34,0,0,1,0,0,0,1,0
4,50000,1,37,0,0,1,0,0,1,0,0
5,50000,0,57,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
29996,220000,0,39,0,0,0,1,0,1,0,0
29997,150000,0,43,0,0,0,1,0,0,1,0
29998,30000,0,37,0,0,1,0,0,0,1,0


In [80]:
#dependent variable
depVar = rawData['default payment next month']
depVar

ID
1        1
2        1
3        0
4        0
5        0
        ..
29996    0
29997    0
29998    1
29999    1
30000    1
Name: default payment next month, Length: 30000, dtype: int64

In [81]:
#Correlation
correlation_df = features.copy(deep=True)
correlation_df['default payment next month'] = depVar
correlation_df.corr()

,LIMIT_BAL,SEX,AGE,edu_0,edu_1,edu_2,edu_3,mar_0,mar_1,mar_2,mar_3,default payment next month
LIMIT_BAL,1.000000,0.024755,0.144713,0.013420,0.258777,-0.147530,-0.139686,-0.011299,0.103701,-0.090998,-0.055806,-0.153520
SEX,0.024755,1.000000,-0.090874,0.008498,-0.022750,0.025353,-0.007650,0.011895,0.030463,-0.031004,-0.001985,-0.039961
AGE,0.144713,-0.090874,1.000000,0.008982,-0.100423,-0.077626,0.231252,0.010731,0.449425,-0.466584,0.082771,0.013890
edu_0,0.013420,0.008498,0.008982,1.000000,-0.092951,-0.117992,-0.055736,-0.005346,0.011299,-0.012420,0.007718,-0.045703
edu_1,0.258777,-0.022750,-0.100423,-0.092951,1.000000,-0.692075,-0.326917,-0.024772,-0.153711,0.164459,-0.043235,-0.051328
edu_2,-0.147530,0.025353,-0.077626,-0.117992,-0.692075,1.000000,-0.414989,-0.030346,0.060923,-0.059692,0.007084,0.036467
edu_3,-0.139686,-0.007650,0.231252,-0.055736,-0.326917,-0.414989,1.000000,0.074668,0.112519,-0.127678,0.043678,0.032403
mar_0,-0.011299,0.011895,0.010731,-0.005346,-0.024772,-0.030346,0.074668,1.000000,-0.038824,-0.045287,-0.004430,-0.013158
mar_1,0.103701,0.030463,0.449425,0.011299,-0.153711,0.060923,0.112519,-0.038824,1.000000,-0.975033,-0.095381,0.029775
mar_2,-0.090998,-0.031004,-0.466584,-0.012420,0.164459,-0.059692,-0.127678,-0.045287,-0.975033,1.000000,-0.111260,-0.030619


In [82]:
#Implement Cross Validation
X_train, X_test, y_train, y_test = train_test_split(features, depVar, test_size = .25, random_state =222)
X_train

,LIMIT_BAL,SEX,AGE,edu_0,edu_1,edu_2,edu_3,mar_0,mar_1,mar_2,mar_3
ID,,,,,,,,,,,
26514,280000,1,63,0,1,0,0,0,1,0,0
6705,130000,0,25,0,0,1,0,0,0,1,0
3223,200000,0,33,0,0,1,0,0,0,1,0
2254,10000,0,48,0,0,0,1,0,0,1,0
29991,140000,0,41,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
15939,170000,1,27,0,0,1,0,0,0,1,0
15887,130000,1,29,0,0,1,0,0,0,1,0
29300,240000,0,26,0,0,1,0,0,0,1,0


In [83]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((22500, 11), (7500, 11), (22500,), (7500,))

In [84]:
#Polynomial features
poly_features = X_train[['LIMIT_BAL']]
poly_features['default payment next month'] = y_train
poly_features_test = X_train[['LIMIT_BAL']]

poly_target = poly_features['default payment next month']

poly_features = poly_features.drop(columns = ['default payment next month'])

# Create the polynomial object with specified degree
poly_transformer = PolynomialFeatures(degree = 3)

# Train the polynomial features
poly_transformer.fit(poly_features)

# Transform the features
poly_features = poly_transformer.transform(poly_features)
poly_features_test = poly_transformer.transform(poly_features_test)
print('Polynomial Features shape: ', poly_features.shape)

poly_transformer.get_feature_names(input_features = ['LIMIT_BAL'])[:15]

# Create a dataframe of the features 
poly_features = pd.DataFrame(poly_features, 
                             columns = poly_transformer.get_feature_names(['LIMIT_BAL']))

# Add in the target
poly_features['default payment next month'] = poly_target

# Find the correlations with the target
poly_corrs = poly_features.corr()['default payment next month'].sort_values()

poly_corrs


Polynomial Features shape:  (22500, 4)


C:\Users\Jeremy\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


LIMIT_BAL                     0.000608
LIMIT_BAL^2                   0.001985
LIMIT_BAL^3                   0.004594
default payment next month    1.000000
1                                  NaN
Name: default payment next month, dtype: float64

In [91]:
#RFE

# create a base classifier used to evaluate a subset of attributes
model = LogisticRegression()
# create the RFE model and select 3 attributes
rfe = RFE(model, 1)
rfe = rfe.fit(X_train, y_train)
# summarize the selection of the attributes
print(rfe.support_)
print(rfe.ranking_)
X_train_RFE = X_train.iloc[:, rfe.get_support()]
X_test_RFE = X_test.iloc[:, rfe.get_support()]

C:\Users\Jeremy\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Jeremy\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Jeremy\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Jeremy\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


[False False False  True False False False False False False False]
[10  6  9  1  5  8  7 11  3  4  2]


C:\Users\Jeremy\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Jeremy\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Jeremy\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Jeremy\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Jeremy\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to sile

In [92]:
#RF
rf = RandomForestClassifier()
rfmodel = rf.fit(X_train, y_train)
rfpreds = rfmodel.predict(X_test)
accuracy_score(y_test, rfpreds)

C:\Users\Jeremy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.7245333333333334

In [93]:
#RF with RFE
rf = RandomForestClassifier()
rfmodel_RFE = rf.fit(X_train_RFE, y_train)
rfpreds_RFE = rfmodel_RFE.predict(X_test_RFE)
accuracy_score(y_test, rfpreds_RFE)

C:\Users\Jeremy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.7756

In [94]:
#N_estimators parameter search
n_estimators=[1,2,3,8,16,32,64,100,200]
for estimator in n_estimators:
    rf = RandomForestClassifier(n_estimators = estimator)
    
    model = rf.fit(X_train, y_train)
    preds = model.predict(X_test)
    print(estimator, accuracy_score(y_test, preds))

1 0.7058666666666666
2 0.7208
3 0.7102666666666667
8 0.7262666666666666
16 0.726
32 0.7281333333333333
64 0.7297333333333333
100 0.7284
200 0.7305333333333334


In [95]:
#max_depth parameter search
max_depths = np.linspace(1, 32, 32, endpoint=True)
for depth in max_depths:
    rf = RandomForestClassifier(n_estimators = 200, max_depth = depth)
    
    model = rf.fit(X_train_RFE, y_train)
    preds = model.predict(X_test_RFE)
    print(depth, accuracy_score(y_test, preds))

1.0 0.7756
2.0 0.7756
3.0 0.7756
4.0 0.7756
5.0 0.7756
6.0 0.7756
7.0 0.7756
8.0 0.7756
9.0 0.7756
10.0 0.7756
11.0 0.7756
12.0 0.7756
13.0 0.7756
14.0 0.7756
15.0 0.7756
16.0 0.7756
17.0 0.7756
18.0 0.7756
19.0 0.7756
20.0 0.7756
21.0 0.7756
22.0 0.7756
23.0 0.7756
24.0 0.7756
25.0 0.7756
26.0 0.7756
27.0 0.7756
28.0 0.7756
29.0 0.7756
30.0 0.7756
31.0 0.7756
32.0 0.7756


In [96]:
#min_samples_splits parameter search
min_samples_splits = np.linspace(0.1, 1.0, 10, endpoint=True)
for min_samples_split in min_samples_splits:
    rf = RandomForestClassifier(n_estimators = 8, max_depth = 24, min_samples_split=min_samples_split)
    
    model = rf.fit(X_train, y_train)
    preds = model.predict(X_test)
    print(min_samples_split, accuracy_score(y_test, preds))

0.1 0.7756
0.2 0.7756
0.30000000000000004 0.7756
0.4 0.7756
0.5 0.7756
0.6 0.7756
0.7000000000000001 0.7756
0.8 0.7756
0.9 0.7756
1.0 0.7756


In [97]:
#min_samples_leafs parameter search
min_samples_leafs = np.linspace(0.1, 0.5, 5, endpoint=True)
for min_samples_leaf in min_samples_leafs:
    rf = RandomForestClassifier(n_estimators = 8, max_depth = 9, min_samples_split = .1, min_samples_leaf=min_samples_leaf)
    
    model = rf.fit(X_train, y_train)
    preds = model.predict(X_test)
    print(min_samples_leaf, accuracy_score(y_test, preds))

0.1 0.7756
0.2 0.7756
0.30000000000000004 0.7756
0.4 0.7756
0.5 0.7756


In [98]:
#rf with tuned parameters
rf = RandomForestClassifier(n_estimators = 8, max_depth = 9, min_samples_split = .1, min_samples_leaf=.1)
rfmodel = rf.fit(X_train, y_train)
rfpreds = rfmodel.predict(X_test)
accuracy_score(y_test, rfpreds)

0.7756

In [107]:
#knn
kn = KNeighborsClassifier()
knmodel = kn.fit(X_train, y_train)
knpreds = knmodel.predict(X_test)
accuracy_score(y_test, knpreds)

0.742

In [106]:
#knn tune Hyperparameters that we want to tune.
leaf_size = list(range(1,50))
n_neighbors = list(range(1,30))
p=[1,2]#Convert to dictionary
parameters = {'n_neighbors':[59,100], 'leaf_size':[45,55], 'p':[1,2]}#Create new KNN object
kn = KNeighborsClassifier()#Use GridSearch
clf = GridSearchCV(kn, parameters)#Fit the model
best_model = clf.fit(X_train,y_train)#Print The value of best Hyperparameters
print('Best leaf_size:', best_model.best_estimator_.get_params()['leaf_size'])
print('Best p:', best_model.best_estimator_.get_params()['p'])
print('Best n_neighbors:', best_model.best_estimator_.get_params()['n_neighbors'])

C:\Users\Jeremy\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Best leaf_size: 45
Best p: 2
Best n_neighbors: 100


In [108]:
#knn tuned parameters
kn = KNeighborsClassifier(n_neighbors = 100, p = 2, leaf_size = 45)
knmodel = kn.fit(X_train, y_train)
knpreds = knmodel.predict(X_test)
accuracy_score(y_test, knpreds)

0.7752

In [118]:
svc = SVC()
svcmodel = svc.fit(X_train, y_train)
svcpreds = svcmodel.predict(X_test)
accuracy_score(y_test, svcpreds)


C:\Users\Jeremy\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.774

In [117]:
np.unique(knpreds, return_counts = True)

(array([0, 1], dtype=int64), array([7473,   27], dtype=int64))